In [14]:
import heapq
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import time

PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

def available_actions(state: np.ndarray) -> list:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state 

RANDOMIZE_STEPS = 100_000
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
state = goal_state.copy()

for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

def hamming_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """Calculates the number of misplaced tiles, excluding the empty tile."""
    return np.sum(state != goal_state) - 1  # Subtract 1 to ignore the empty tile

def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """Calculates the sum of Manhattan distances of tiles from their goal positions."""
    total_distance = 0
    for val in range(1, PUZZLE_DIM**2):
        current_pos = np.argwhere(state == val)[0]
        goal_pos = np.argwhere(goal_state == val)[0]
        total_distance += abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1])
    return total_distance

def solve_n2_1_puzzle(initial_state: np.ndarray, goal_state: np.ndarray, heuristic_func, length_criteria=0) -> list:
    """Solves the n^2-1 puzzle using A* search with the given heuristic function."""
    pq = []
    heapq.heappush(pq, (heuristic_func(initial_state, goal_state), 0, [], initial_state))
    visited = set()
    start_time = time.time()
    min_distance = float('inf')
    best_state = None

    while pq:
        _, move_count, path, current_state = heapq.heappop(pq)

        if np.array_equal(current_state, goal_state):
            return path, current_state

        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_path = path + [act]
            new_priority = heuristic_func(new_state, goal_state) + length_criteria * len(new_path)
            heapq.heappush(pq, (new_priority, len(new_path), new_path, new_state))

            current_distance = heuristic_func(new_state, goal_state)
            if current_distance < min_distance:
                min_distance = current_distance
                best_state = new_state

        if time.time() - start_time >= 5:
            print(f"Current depth: {move_count}, queue size: {len(pq)}, current min distance: {min_distance}")
            start_time = time.time()

    print(f"Best state with min distance: {min_distance}")
    print(best_state)
    return None

print("Solving...")

# solution, final_state = solve_n2_1_puzzle(state, goal_state, manhattan_distance, length_criteria=0.05*6)
solution, final_state = solve_n2_1_puzzle(state, goal_state, manhattan_distance, length_criteria=0.05*0)
if solution:
    print(f"Solution found in {len(solution)} moves.")
    print(final_state)
    print(solution)
else:
    print("No solution found.")

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 71704.07it/s]


Solving...
Current depth: 145, queue size: 2362, current min distance: 14
Current depth: 201, queue size: 4615, current min distance: 8
Current depth: 198, queue size: 6571, current min distance: 8
Current depth: 216, queue size: 8330, current min distance: 8
Current depth: 180, queue size: 9447, current min distance: 8
Current depth: 191, queue size: 10623, current min distance: 8
Current depth: 216, queue size: 12602, current min distance: 8
Current depth: 202, queue size: 14457, current min distance: 4
Current depth: 229, queue size: 15988, current min distance: 4
Current depth: 208, queue size: 17516, current min distance: 4
Current depth: 239, queue size: 19241, current min distance: 4
Current depth: 257, queue size: 21142, current min distance: 4
Current depth: 240, queue size: 23044, current min distance: 4
Current depth: 269, queue size: 24694, current min distance: 4
Solution found in 300 moves.
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23